### 导入常量

In [ ]:
import json

# 从 JSON 文件中读取常量数据
with open('constants.json', 'r') as json_file:
    constants = json.load(json_file)

## 一、数据集预处理

### 1、标记
将训练集的mentions转换为text对应的标记

In [ ]:
import pandas as pd

# 读取训练集
df = pd.read_excel("data/train.xlsx")

# entity的起始和后续标记
B_ENT = constants["B_ENT"]
I_ENT = constants["I_ENT"]

# 逐个样本处理
for index, row in df.iterrows():

    # mentions的列表
    mention_list = eval(row['mentions'])

    # text字符串的长度
    text_len = len(row['text'])

    # 创建与text等长的label
    label = [0]*text_len

    # 对mention列表的每一个mention做处理，其中每个mention是一个dict
    for mention in mention_list:

        # mention字典中的'offset'字段表示某个实体的开始和结束位置，左闭右开
        (start, end) = eval(mention['offset'])

        # entity的起始位置标记为B_ENT
        label[start] = B_ENT
        
        # entity的其余位置标记为I_ENT
        for num in range(start+1, end):
            label[num] = I_ENT

    # 存储label
    df.loc[index, "labels"] = str(label)
    
# 保存为新的CSV文件
df.to_csv("data/train_labeled.csv")


### 2、分句
将大段新闻分割为短句，便于训练

#### 方案一：用句号划分

In [ ]:
# df0 = pd.read_csv('data/train_labeled.csv')

# # 以句号为分隔
# char_sep = '。'
# li = []

# for index, row in df0.iterrows():
#     # indices是分隔的位置list，第一句从头开始
#     indices = [-1]

#     text = row['text']
#     text_len = len(text)

#     # 找到所有的句号
#     for i in range(len(text)):
#         if text[i] == char_sep:
#             indices.append(i)

#     # 如果结尾不是句号，添加最后一个字符为分隔
#     if indices[-1] != text_len - 1:
#         indices.append(text_len - 1)

#     # 逐句处理
#     for j in range(len(indices)-1):
#         begin = indices[j] + 1
#         end = indices[j+1] + 1
#         sent = text[begin: end]
#         label = eval(row['labels'])[begin: end]
#         length = end - begin
#         li.append([length, sent, label])
    
# df = pd.DataFrame(li, columns = ['length', 'text', 'labels'])

# df.to_csv('data/sep_by_。.csv')


#### 方案二：指定最大长度

In [ ]:
# df = pd.read_csv('data/train_labeled.csv')
# df_s = pd.DataFrame(columns = ['segment_number','id','segment_len','text','mentions','labels'])
# max_size = 100
# #句段长度上限
# #print(df_s.shape)
# #header = 0 把第一行作为列名称，首位空缺设置为默认的0
# #print(df.loc[0])
# #显示列标
# #预期效果：每行是一个短于max_size样本片段，一个样本占有若干行（若干片段）。
# #预期效果：每列是一个片段的信息：
# #         列标：0,               1，        2，      3，      4，          5，          
# #              所属样本第几个片段 所属样本id 片段长度 片段文本  片段mentions  片段labels
# seg_num = 0
# #总片段下标

# for row in df.values:
#     #print(type(row),row.shape)
#     mention_list = eval(row[3])
#     #print(eval(row[5]))
#     #al_len = 0
#     al_seg_len = 0
#     segs = 0
#     #该文本已经切割的片段总长度和片段数目
    
#     df_s.loc[seg_num] = [segs,row[1],0,'',[],'']
#     #df_s = df_s.append(pd.DataFrame([[segs,row[1],0,'',[],'']],columns = df_s.columns))
#     #print(type(df_s.loc[seg_num]))
#     #初始化该文本第一个片段
#     for i in range(len(mention_list)):
#     # 对mention列表的每一个mention做处理，其中每个mention是一个dict
        
#         start, end = eval(mention_list[i]['offset'])
#         # mention字典中的'offset'字段表示某个实体的开始和结束位置，左闭右开
#         if end < al_seg_len + max_size:
#         #实体全部在长度范围内
#             df_s.iloc[seg_num,4].append(mention_list[i])
#             #print(mention_list[i])
#             if i == len(mention_list):
#             #最后一个mention
#                 df_s.iloc[seg_num,3]=row[2][al_seg_len:min(al_seg_len + max_size,len(row[2]))]
#                 df_s.iloc[seg_num,5]=str(eval(row[5])[al_seg_len:min(al_seg_len + max_size,len(eval(row[5])))])
#                 al_seg_len = min(al_seg_len + max_size,len(row[2]))

#                 df_s.iloc[seg_num,2]=len(df_s.iloc[seg_num,3])
#                 seg_num +=1   
#                 segs += 1
#         else:
#         #过长，该实体放入下一个片段,开始整理已经确定的片段
#             i -= 1
#             #下一次还要放入该mention
#             df_s.iloc[seg_num,3]=row[2][al_seg_len:min(al_seg_len + max_size,start)]
#             df_s.iloc[seg_num,5]=str(eval(row[5])[al_seg_len:min(al_seg_len + max_size,start)])
            
#             al_seg_len = min(al_seg_len + max_size,start)

#             df_s.iloc[seg_num,2]=len(df_s.iloc[seg_num,3])
#             seg_num +=1   
#             segs += 1
#             if al_seg_len < len(row[2]): df_s.loc[seg_num] = [segs,row[1],0,'',[],'']
#             #新片段
#     #最后处理尾部无mentions区域
#     while al_seg_len < len(row[2]):
#         df_s.iloc[seg_num,3]=row[2][al_seg_len:min(al_seg_len + max_size,len(row[2]))]
#         df_s.iloc[seg_num,5]=str(eval(row[5])[al_seg_len:min(al_seg_len + max_size,len(eval(row[5])))])
#         al_seg_len = min(al_seg_len + max_size,len(row[2]))
#         if len(eval(df_s.iloc[seg_num,5])) != len(df_s.iloc[seg_num,3]) or len(eval(df_s.iloc[seg_num,5]))>max_size: 
#            print(len(df_s.iloc[seg_num,3]),len(eval(df_s.iloc[seg_num,5])))
#         df_s.iloc[seg_num,2]=len(df_s.iloc[seg_num,3])
#         seg_num +=1   
#         segs += 1
#         if al_seg_len < len(row[2]): df_s.loc[seg_num] = [segs,row[1],0,'',[],'']
#     #print(df_s.loc[0])

# df_s.to_csv("data/train_separated.csv")

#### 方案三：用多种标点划分

In [ ]:
# 句子的最大长度
max_size = constants["max_size"]

# 以句号为分隔
char_sep = '。'
sep_list1 = [',',' ',';']
sep_list2 = ['、',':',')']

def find_sep(text):

    """找到所有划分位置"""

    # 文本长度
    text_len = len(text)

    # indices是分隔的位置list，第一句从头开始
    indices = [-1]

    # 找到所有用于划分的标点符号
    for i in range(text_len):

        # 首先尝试找句号
        if text[i] == char_sep:
            indices.append(i)
            continue

        # 指定长度内没有句号
        if i - indices[-1] >= max_size - 2:

            # flag表示是否找到sep_list1中的标点
            flag = 0

            # 尝试找sep_list1中的标点
            for j in range (i, indices[-1], -1):
                if text[j] in sep_list1:
                    indices.append(j)
                    flag = 1
                    break
             
            if flag == 0:

                # 最后尝试找sep_list2中的标点
                for j in range (i, indices[-1], -1):
                    if text[j] in sep_list2:
                        indices.append(j)
                        break

    # 如果结尾不是句号，添加最后一个字符为分隔
    if indices[-1] != text_len - 1:
        indices.append(text_len - 1)

    return indices


def generate_data(indices):

    """逐句处理，将分割后的句子及标记存储至data"""

    for j in range(len(indices)-1):

        begin = indices[j] + 1
        end = indices[j+1] + 1
        length = end - begin

        sent = text[begin: end]
        sent = replace_space(sent)

        label = eval(row['labels'])[begin: end]
        
        # 去除太短的句子
        if length >= 10:
            data.append([length, sent, label])


def replace_space(sent):

    """由于tokenizer编码时会省略空格，因此将空格替换为其无法识别的省略号"""

    return sent.replace(" ", "…")


# 带标记的数据
df0 = pd.read_csv('data/train_labeled.csv')

# 新的数据
data = []

# 逐行处理
for index, row in df0.iterrows():

    #文本
    text = row['text']

    #分割位置
    indices = find_sep(text)

    # 根据分割位置生成data
    generate_data(indices)
    
# 保存为DataFrame
df = pd.DataFrame(data, columns = ['length', 'text', 'labels'])

# 按句子长度倒序排列
df = df.sort_values(by='length', ascending=False)

# 保存
df.to_csv('data/sep_max_'+str(max_size)+'.csv')


## 二、数据处理

### 1、加载分词器并测试

In [ ]:
from transformers import AutoTokenizer

#加载分词器
tokenizer = AutoTokenizer.from_pretrained('hfl/rbt6',pad_to_max_length=False)

# 测试文本
text = ["红土创新基金管理有限公司6月30日发布公告，红土创新盐田港REIT(180301)的基金经理新聘陈超。"
,"金能科技: 金能科技股份有限公司关于“金能转债”转股价格调整的提示性公告"]

# 将文本进行逐字分解
def split_sent(text):
    return [char for char in text]


def split(text_list):

    text_split = []
    for sent in text_list:
        sent_split = split_sent(sent)
        text_split.append(sent_split)
    return text_split


# 分解并编码
def split_and_encode(text_list):

    inputs = tokenizer.batch_encode_plus(
    split(text_list),
    truncation=True,
    padding=True,
    return_tensors='pt',
    is_split_into_words=True)

    return inputs

# 展示编码结果
inputs_test = split_and_encode(text)
for key,value in inputs_test.items():
    print(key ,": \n", value, "\n")


### 2、定义新的Dataset类

In [ ]:
import torch
import datasets

class Dataset(torch.utils.data.Dataset):

    """Dataset类，用于处理训练集"""

    def __init__(self, path):
        
        # 用pandas读取csv，并将其转换为Dataset
        data_df = pd.read_csv(path)
        data_list = data_df.to_dict(orient="list")
        dataset = datasets.Dataset.from_dict(data_list)

        #过滤掉太长的句子
        def f(data):
            return data['length'] <= max_size - 2

        dataset = dataset.filter(f)

        self.data = dataset

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        token = split_sent(self.data[index]['text'])
        label = eval(self.data[index]['labels'])

        return token, label 

# 创建数据集实例
dataset = Dataset('data/sep_max_'+str(max_size)+'.csv')

# 使用数据集
token, label = dataset[0]

# 展示
len(dataset), list(token), label

### 3、定义数据整理函数

In [ ]:

pad = constants["pad"]

#数据整理函数
def collate_fn(data):
    
    # tokens是分解后的文本
    tokens = [row[0] for row in data]
    # labels是对应的标记
    labels = [row[1] for row in data]

    # inputs是tokens的编码
    inputs = tokenizer.batch_encode_plus(tokens,
                                         truncation=True,
                                         padding=True,
                                         return_tensors='pt',
                                         is_split_into_words=True)

    # 编码的长度（最长长度）
    length = inputs['input_ids'].shape[1]

    # 将[CLS]和[PAD]标记，并将标记长度与编码长度统一
    for i in range(len(labels)):
        labels[i] = [pad] + labels[i]
        labels[i] += [pad] * length
        labels[i] = labels[i][:length]

    return inputs, torch.LongTensor(labels)


### 4、定义数据加载器

In [ ]:
# 批大小
batch_size = constants["batch_size"]

#数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=batch_size,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)

#查看数据样例
for i, (inputs, labels) in enumerate(loader):
    break

print(len(loader))
print(len(labels))
print(tokenizer.decode(inputs['input_ids'][0]))

for k, v in inputs.items():
    print(k,'\t', v.shape)

## 三、模型构建

### 1、加载预训练模型

In [ ]:
from transformers import AutoModel

# 加载预训练模型
pretrained = AutoModel.from_pretrained('hfl/rbt6')

# 统计参数量
print(sum(i.numel() for i in pretrained.parameters()) / 10000, '万')

#模型试算
pretrained(**inputs).last_hidden_state.shape

### 2、定义下游主体模型

In [ ]:
import torch

out_len = constants["out_len"]

# 定义运行的设备
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

device1 = torch.device("cpu")


class Model(torch.nn.Module):

    """下游命名实体识别模型"""

    def __init__(self):

        super().__init__()

        # tuning用于调整模式，False表示使用外部预训练模型，True表示使用本模型
        self.tuneing = False

        # 模型初始化
        self.pretrained = None

        # 添加RNN层
        self.rnn = torch.nn.GRU(768, 768, batch_first=True)

        # 添加全连接层
        self.fc = torch.nn.Linear(768, out_len)


    # 传播函数
    def forward(self, inputs):

        if self.tuneing:
            out = self.pretrained(**inputs).last_hidden_state
        else:
            with torch.no_grad():
                out = pretrained(**inputs).last_hidden_state

        out, _ = self.rnn(out)

        out = self.fc(out).softmax(dim=2)

        return out


    # 调整tuning的函数
    def fine_tuneing(self, tuneing):

        self.tuneing = tuneing

        if tuneing:
            for i in pretrained.parameters():
                i.requires_grad = True

            pretrained.train()
            self.pretrained = pretrained

        else:
            for i in pretrained.parameters():
                i.requires_grad_(False)

            pretrained.eval()
            self.pretrained = None


pretrained = pretrained.to(device)

# 创建模型实例
model = Model()

inputs = inputs.to(device)
model.to(device)



model(inputs).shape

### 3、定义辅助函数

#### （1）对计算结果和label变形,并且移除pad

In [ ]:
def reshape_and_remove_pad(outs, labels, attention_mask):

    #变形,便于计算loss
    #[b, lens, out_len] -> [b*lens, out_len]
    outs = outs.reshape(-1, out_len)
    
    #[b, lens] -> [b*lens]
    labels = labels.reshape(-1)

    #忽略对pad的计算结果
    #[b, lens] -> [b*lens - pad]
    select = attention_mask.reshape(-1) == 1
    outs = outs[select]
    labels = labels[select]

    return outs, labels

#### （2）获取识别正确数量和总数，包括总体和除零两种情况

In [ ]:
def get_correct_and_total_count(labels, outs):

    #[b*lens, out_len] -> [b*lens]
    outs = outs.argmax(dim=1)
    correct = (outs == labels).sum().item()
    total = len(labels)

    # 计算除了0以外元素的正确率,因为0太多了,包括的话,正确率很容易虚高
    select = (labels == B_ENT) + (labels == I_ENT)

    outs = outs[select]
    labels = labels[select]

    correct_content = (outs == labels).sum().item()
    total_content = len(labels)

    print('正确数:', correct, '总数:', total, '去零正确数:', correct_content, '去零总数:', total_content)

    return correct, total, correct_content, total_content

# 测试
get_correct_and_total_count(torch.ones(16), torch.randn(16, out_len))

## 四、模型训练

### 1、定义训练函数

In [ ]:
# 导入优化器
from torch.optim import AdamW

def train(epochs):

    """训练函数"""

    # 学习率
    lr = 2e-5 if model.tuneing else 5e-4

    # 优化器
    optimizer = AdamW(model.parameters(), lr=lr)

    # 损失函数
    criterion = torch.nn.CrossEntropyLoss()

    model.train()

    for epoch in range(epochs):

        for step, (inputs, labels) in enumerate(loader):

            inputs = inputs.to(device)

            #模型计算
            #[b, lens] -> [b, lens, out_len]
            outs = model(inputs)

            labels = labels.to(device)

            #对outs和label变形,并且移除pad
            #outs -> [b, lens, out_len] -> [c, out_len]
            #labels -> [b, lens] -> [c]
            outs, labels = reshape_and_remove_pad(outs, labels, inputs['attention_mask'])

            #梯度下降
            loss = criterion(outs, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            # 每50步输出一次训练效果
            if step % 50 == 0:
                counts = get_correct_and_total_count(labels, outs)

                accuracy = counts[0] / counts[1]
                accuracy_content = counts[2] / counts[3]

                print('轮数:', epoch, '批数:', step, '损失:', loss.item(), '准确率:', accuracy, '去零准确率:', accuracy_content)

        # 模型保存
        torch.save(model, 'model/model1.model')

### 2、非tuning训练

In [ ]:
model.fine_tuneing(False)
print(sum(p.numel() for p in model.parameters()) / 10000)
train(1)

### 3、tuning训练

In [ ]:
model.fine_tuneing(True)
print(sum(p.numel() for p in model.parameters()) / 10000)
train(10)

### 4、效果测试

In [ ]:
def test():

    """测试函数"""

    # 导入训练好的模型
    model_load = torch.load('model/model.model')
    model_load.eval()

    # 数据加载器
    loader_test = torch.utils.data.DataLoader(dataset=dataset,
                                              batch_size=128,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)

    # 变量初始化
    correct = 0
    total = 0
    correct_content = 0
    total_content = 0

    for step, (inputs, labels) in enumerate(loader_test):

        # 测试5批
        if step == 5:
            break

        print('批数:', step)

        with torch.no_grad():

            inputs = inputs.to(device)
            model_load = model_load.to(device)

            #[b, lens] -> [b, lens, out_len] -> [b, lens]
            outs = model_load(inputs)
            
            labels = labels.to(device)

        #对outs和label变形,并且移除pad
        #outs -> [b, lens, out_len] -> [c, out_len]
        #labels -> [b, lens] -> [c]
        outs, labels = reshape_and_remove_pad(outs, labels, inputs['attention_mask'])

        # 获取识别正确数量和总数
        counts = get_correct_and_total_count(labels, outs)

        correct += counts[0]
        total += counts[1]
        correct_content += counts[2]
        total_content += counts[3]

    print('总准确率:', correct / total,'去零总准确率:', correct_content / total_content)


test()

## 五、模型预测

### 1、手动预测

In [ ]:
import torch

def predict_one():

    # 导入训练好的模型
    model_load = torch.load('model/model.model')
    model_load.eval()


    # 自定义文本
    text = ["红土创新基金管理有限公司6月30日发布公告，红土创新盐田港REIT(180301)的基金经理新聘陈超。"
    ,"金能科技: 金能科技股份有限公司关于“金能转债”转股价格调整的提示性公告",
    "大地熊成功收购技研株式会社安徽大地熊新材料股份有限公司2023年6月22日,大地熊日本株式会社与技研株式会社"\
        "(英文名:p.m.giken inc,简称pm公司)原有股东和相关方签订收购合同》,大地熊日本株式会社全资收购pm公司"\
        "并即时启动相关手续交接。 pm公司是专业从事磁钢整体磁气回路的解析与设计,注塑磁体的开发、制造与销售。"\
        "此次收购是大地熊向注塑磁体领域的全新产业布局,将进一步推进大地熊海内外战略合作与业务拓展,丰富公司磁性材料及部件产品种类,提升公司综合实力和整体竞争力。",
    "黑猫股份:设立全资子公司投建年产16万吨碳材/橡胶复合母胶项目证券时报e公司讯,黑猫股份(002068)6月27日晚间"\
    "公告,公司拟出资1亿元在辽宁省朝阳市高新技术产业开发区设立全资子公司辽宁黑猫复合新材料科技有限公司(简称辽"\
    "宁黑猫),并且将以辽宁黑猫为项目主体,投资新建年产16万吨碳材/橡胶复合母胶项目,分三期进行建设,项目预计投资总额为6.88亿元。",
    "秋田满满、小鹿蓝蓝食安问题频发,投诉也没用随着我国新生代父母的科学喂养观念加强,我国婴童辅食行业消费规模"\
        "持续上升。早在2019年我国婴幼儿辅食消费市场规模就已经达到404亿元,年复合增长率高达23%,预计未来我国婴幼儿辅食市场规模应在千亿以上。",
        "非银金融600218全柴动力46875.42 13.57 -1.30汽车002166莱茵生物46891.42 13.40 -3.47"]
    
    text_split = []
    for sent in text:
        sent_split = [char for char in sent]
        text_split.append(sent_split)

    inputs = tokenizer.batch_encode_plus(
    text_split,
    truncation=True,
    padding=True,
    return_tensors='pt',
    is_split_into_words=True)


    with torch.no_grad():
        #[b, lens] -> [b, lens, 8] -> [b, lens]
        inputs = inputs.to(device)
        outs = model_load(inputs).argmax(dim=2)

    for i in range(len(text)):
        #移除pad
        select = inputs['attention_mask'][i] == 1
        input_id = inputs['input_ids'][i, select]
        out = outs[i, select]

        #输出原句子
        print(tokenizer.decode(input_id).replace(' ', ''), len(input_id))

        #输出tag
        s = ''
        for j in range(len(out)):
            if out[j] == 0:
                s += '·'
                continue
            s += tokenizer.decode(input_id[j])
            s += str(out[j].item())

        print(s)
        print('==========================')
    
predict_one()

### 2、测试集预测

In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer

#加载分词器
tokenizer = AutoTokenizer.from_pretrained('hfl/rbt6')

# 分句
def separate(text):

    sent_list = []

    indices = find_sep(text)

    for j in range(len(indices)-1):

        begin = indices[j] + 1
        end = indices[j+1] + 1
        sent = text[begin: end]

        sent_list.append(sent)

    return sent_list


# 将完整的label和text转换为mentions
def label_to_mention(text, labels):
    
    mention_list = []

    length = len(labels)

    for index in range(length):

        if labels[index] == B_ENT:
            begin = index
            entity_index = index

            while labels[entity_index] != 0 and entity_index < length:
                entity_index += 1

            end = entity_index
    
            entity = text[begin: end]

            mention = {"mention": entity, "offset": (begin, end)}

            mention_list.append(mention)

    return mention_list


# 预测函数
def predict():

    # 导入模型
    model_load = torch.load('model/model.model')
    model_load.eval()

    # 导入测试集
    data = pd.read_excel('data/test.xlsx')

    for index, row in data.iterrows():

        print(index)

        text = row['text']

        # 分句
        sent_list = separate(text)
        labels = []

        # 分词
        text_split = []
        for sent in sent_list:
            sent = replace_space(sent)
            sent_split = [char for char in sent]
            text_split.append(sent_split)

        # 编码
        inputs = tokenizer.batch_encode_plus(
            text_split,
            truncation=True,
            padding=True,
            return_tensors='pt',
            is_split_into_words=True)

        # 计算
        with torch.no_grad():
            #[b, lens] -> [b, lens, 8] -> [b, lens]
            inputs = inputs.to(device)
            outs = model_load(inputs).argmax(dim=2)
            inputs = inputs.to(device1)
            outs = outs.to(device1)

        # 标记处理
        for i in range(len(text_split)):
            #移除pad, cls和sep
            select = inputs['attention_mask'][i] == 1
            out = outs[i, select]
            label = out.tolist()[1: len(out)-1]
            labels += label

        # 保存labels和mentions
        data.loc[index, "labels"] = str(labels)
        mentions = label_to_mention(text, labels)
        data.loc[index, "mentions"] = str(mentions)

    # 保存为文件
    data.to_csv('data/out.csv')


predict()

## 六、知识库查询

In [ ]:
import pandas as pd
import math

# 加载测试集预测结果
df = pd.read_csv('data/out.csv')

# 加载知识库
kb = pd.read_excel('data/knowledge_base_new.xlsx')

# 在知识库中查找并将知识添加到knowledge_list中
def find(entity, offset):
    for index, row in kb.iterrows():
        if index not in index_list and (row['stockName'] == entity or row['companyName'] == entity):
            know = \
            {"companyId": row['companyId'],
            "companyName": row['companyName'],
            "mention": entity, 
            "offset": offset, 
            "stockId": int(row['stockId']) if not math.isnan(row['stockId']) else 'null', 
            "stockName": row['stockName'] if not math.isnan(row['stockId']) else 'null', 
            "type": row['type'][:2]}
            knowledge_list.append(know)
            index_list.append(index)
        

for index, row in df.iterrows():

    print(index)

    mention_list = eval(row['mentions'])

    # 存储找过的entity，避免重复
    entity_list = []

    # 存储已找到的knowledge下标，避免重复
    index_list = []

    # 用于存储找到的knowledge
    knowledge_list = []

    # 对mention列表的每一个mention做处理，其中每个mention是一个dict
    for mention in mention_list:

        entity = mention['mention']

        offset = mention['offset']

        if entity not in entity_list:

            find(entity, offset)

            entity_list.append(entity)

    # 存储结果
    df.loc[index, "result"] = str(knowledge_list)

# 保存为文件
df.to_csv('data/knowledge_test.csv')